In [677]:
import pandas as pd
import numpy as np

In [678]:
df = pd.read_csv('resume_data.csv')

Data Cleaning

getting the experience years by considering the start and end dates

droping the unwanted columns

In [682]:
df = df.drop([
    'address', 'career_objective', 'professional_company_names', 'company_urls', 'positions', 'locations', 'responsibilities', 'extra_curricular_activity_types',
    'extra_curricular_organization_names', 'extra_curricular_organization_links', 'role_positions', 
    'languages', 'proficiency_levels', 'certification_providers', 'online_links', 'issue_dates', 
    'expiry_dates', 'responsibilities.1'
], axis=1)

In [683]:
df.head()

,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,start_dates,end_dates,related_skils_in_job,certification_skills,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,skills_required,matched_score
0,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Nov 2019'],['Till Date'],[['Big Data']],NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,NaN,0.850000
1,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['Sep 2019'],['Till Date'],"[['Data Analysis', 'Business Analysis', 'Machi...",NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),NaN,NaN,0.750000
2,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],['June 2018'],['Till Date'],"[['Unified Payment Interface', 'Risk Predictio...",NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,NaN,Brand Promotion\nCampaign Management\nField Su...,0.416667
3,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],['Computer Applications'],"['January 2011', 'January 2008', 'January 2006...","['November 2015', 'January 2010', 'January 200...","[['accounts receivables', 'banking', 'G/L Acco...",NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Age 22 to 30 years,Fast typing skill\nIELTSInternet browsing & on...,0.760000
4,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],['Accounting'],"['January 2016', 'January 2016', 'January 2012...","['Current', 'January 2016', 'January 2015', 'J...","[['collections', 'accounts receivable', 'finan...",[None],Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS\niOS App Developer\niOS Application Develo...,0.650000


clearing N/A 's from dates arrays

In [687]:
import ast

def remove_na_and_none_from_list(col):
    def clean(entry):
        try:
            # if already a list, skip parsing
            if isinstance(entry, list):
                items = entry
            else:
                items = ast.literal_eval(entry)
            return [i for i in items if str(i).strip().upper() not in ['N/A', 'NONE'] and i is not None]
        except:
            return entry  # return original if parsing fails
    return col.apply(clean)

df['start_dates'] = remove_na_and_none_from_list(df['start_dates'])
df['end_dates'] = remove_na_and_none_from_list(df['end_dates'])


Convert seasons('Summer', 'Fall',....) into months

In [690]:
import re

def replace_seasons_with_months(col):
    # Mapping of season to month
    season_to_month = {
        'spring': '03',
        'summer': '06',
        'fall': '09',
        'autumn': '09',
        'winter': '12',
    }

    def replace_season(entry):
        try:
            items = ast.literal_eval(entry) if isinstance(entry, str) else entry
            updated_items = []
            for item in items:
                if item is None:
                    updated_items.append(item)
                    continue
                # Match season followed by year, e.g., "Summer 2013"
                match = re.match(r'(?i)\b(spring|summer|fall|autumn|winter)\b\s+(\d{4})', str(item).strip())
                if match:
                    season = match.group(1).lower()
                    year = match.group(2)
                    month = season_to_month.get(season)
                    updated_items.append(f"{month}/{year}")
                else:
                    updated_items.append(item)
            return updated_items
        except:
            return entry  # keep original if parsing fails

    return col.apply(replace_season)

df['start_dates'] = replace_seasons_with_months(df['start_dates'])
df['end_dates'] = replace_seasons_with_months(df['end_dates'])



replace 'till date', 'current', 'present', 'ongoing', '∞' with today date

In [693]:
from datetime import datetime
import ast

def replace_current_terms_with_today(col):
    today_str = datetime.today().strftime('%b %d %Y')  # Example: 'Apr 22 2025'
    keywords = {'till date', 'current', 'ongoing', 'present', '∞'}

    def clean(entry):
        try:
            items = ast.literal_eval(entry) if isinstance(entry, str) else entry
            return [
                today_str if isinstance(i, str) and i.strip().lower() in keywords else i
                for i in items
            ]
        except:
            return entry  # return as-is if parsing fails

    return col.apply(clean)

# Apply it
df['start_dates'] = replace_current_terms_with_today(df['start_dates'])
df['end_dates'] = replace_current_terms_with_today(df['end_dates'])


filter and display rows where either start_dates or end_dates columns contains any item with the substring"20XX"

In [696]:
import ast

def contains_20xx(entry):
    try:
        items = ast.literal_eval(entry) if isinstance(entry, str) else entry
        return any(isinstance(i, str) and '20xx' in i.lower() for i in items)
    except:
        return False

mask = df['start_dates'].apply(contains_20xx) | df['end_dates'].apply(contains_20xx)
df_with_20xx = df.loc[mask, ['start_dates', 'end_dates']]

print(df_with_20xx)


                 start_dates                         end_dates
81    [May 20XX, April 20XX]           [August 20XX, May 20XX]
180       [20xx, 20xx, 20xx]  [Apr 23 2025, Apr 23 2025, 20xx]
222   [May 20XX, April 20XX]           [August 20XX, May 20XX]
548       [20xx, 20xx, 20xx]  [Apr 23 2025, Apr 23 2025, 20xx]
632         [September 20XX]                      [April 20XX]
...                      ...                               ...
9267        [September 20XX]                      [April 20XX]
9324        [September 20XX]                      [April 20XX]
9373  [May 20XX, April 20XX]           [August 20XX, May 20XX]
9453        [September 20XX]                      [April 20XX]
9508      [20xx, 20xx, 20xx]  [Apr 23 2025, Apr 23 2025, 20xx]

[84 rows x 2 columns]


replace them with 'unknown'

In [697]:
import ast

def replace_array_with_unknown(entry):
    try:
        items = ast.literal_eval(entry) if isinstance(entry, str) else entry
        if any(isinstance(i, str) and '20xx' in i.lower() for i in items):
            return 'unknown'  # Replace entire array with the string 'unknown'
        return items
    except:
        return entry

df['start_dates'] = df['start_dates'].apply(replace_array_with_unknown)
df['end_dates'] = df['end_dates'].apply(replace_array_with_unknown)


replace each record in start_dates and end_dates with 'unknown' if either one of them contains an empty array.

In [700]:
import ast

def replace_empty_array_with_unknown(row):
    try:
        # Parse the entries if they are string representations of lists
        start = ast.literal_eval(row['start_dates']) if isinstance(row['start_dates'], str) else row['start_dates']
        end = ast.literal_eval(row['end_dates']) if isinstance(row['end_dates'], str) else row['end_dates']
        
        # Check if either start or end is an empty list
        if isinstance(start, list) and len(start) == 0 or isinstance(end, list) and len(end) == 0:
            return pd.Series({'start_dates': 'unknown', 'end_dates': 'unknown'})
        else:
            return pd.Series({'start_dates': row['start_dates'], 'end_dates': row['end_dates']})
    except:
        # In case of any parsing error, return the original values
        return pd.Series({'start_dates': row['start_dates'], 'end_dates': row['end_dates']})

# Apply the function to each row
df[['start_dates', 'end_dates']] = df.apply(replace_empty_array_with_unknown, axis=1)


converting all dates in string format to YYYY-MM

In [705]:
import numpy as np
import pandas as pd
from dateutil import parser
import ast

def standardize_date_array(entry):
    if isinstance(entry, str) and entry == 'unknown':
        return entry
    if isinstance(entry, float) and pd.isna(entry):
        return entry

    try:
        items = ast.literal_eval(entry) if isinstance(entry, str) else entry
        if not isinstance(items, list):  # guard clause
            return entry

        cleaned = []
        for item in items:
            if isinstance(item, str):
                try:
                    parsed = parser.parse(item, fuzzy=True)
                    cleaned.append(parsed.strftime('%Y-%m'))
                except:
                    cleaned.append(item)
            else:
                cleaned.append(item)
        return cleaned
    except:
        return entry
    
df['start_dates'] = df['start_dates'].apply(standardize_date_array)
df['end_dates'] = df['end_dates'].apply(standardize_date_array)


check if there are arrays with unequal length

In [707]:
import ast

def count_unequal_length_date_arrays(df):
    count = 0
    for start, end in zip(df['start_dates'], df['end_dates']):
        if isinstance(start, list) and isinstance(end, list):
            if len(start) != len(end):
                count += 1
    return count

unequal_length_count = count_unequal_length_date_arrays(df)
print("Total records with unequal-length date arrays:", unequal_length_count)


Total records with unequal-length date arrays: 452


replace the entries having unequal array length with string 'unknown'.

In [708]:
def replace_unequal_length_records(df):
    for idx, (start, end) in enumerate(zip(df['start_dates'], df['end_dates'])):
        if isinstance(start, list) and isinstance(end, list):
            if len(start) != len(end):
                df.at[idx, 'start_dates'] = 'unknown'
                df.at[idx, 'end_dates'] = 'unknown'
    return df

# Apply the function to your dataframe
df = replace_unequal_length_records(df)


sorting the dates arrays in accending order in both columns

In [710]:
from datetime import datetime

def sort_dates_in_array(entry):
    try:
        # Convert string dates to datetime objects for sorting
        items = ast.literal_eval(entry) if isinstance(entry, str) else entry
        # Sort the items based on datetime conversion
        sorted_items = sorted(items, key=lambda x: datetime.strptime(x, '%b %Y') if ' ' in x else datetime.strptime(x, '%m/%Y') if '/' in x else datetime.strptime(x, '%Y-%m'))
        return sorted_items
    except:
        return entry  # If any error occurs, return the original entry

# Apply the sorting function to both start_dates and end_dates
df['start_dates'] = df['start_dates'].apply(sort_dates_in_array)
df['end_dates'] = df['end_dates'].apply(sort_dates_in_array)
from datetime import datetime

def sort_dates_in_array(entry):
    try:
        # Convert string dates to datetime objects for sorting
        items = ast.literal_eval(entry) if isinstance(entry, str) else entry
        # Sort the items based on datetime conversion
        sorted_items = sorted(items, key=lambda x: datetime.strptime(x, '%b %Y') if ' ' in x else datetime.strptime(x, '%m/%Y') if '/' in x else datetime.strptime(x, '%Y-%m'))
        return sorted_items
    except:
        return entry  # If any error occurs, return the original entry

# Apply the sorting function to both start_dates and end_dates
df['start_dates'] = df['start_dates'].apply(sort_dates_in_array)
df['end_dates'] = df['end_dates'].apply(sort_dates_in_array)


In [713]:
def calculate_experience(start_dates, end_dates):
    # Check for 'unknown' in strings
    if isinstance(start_dates, str) and start_dates.lower() == 'unknown':
        return 'unknown'
    if isinstance(end_dates, str) and end_dates.lower() == 'unknown':
        return 'unknown'

    # Check for 'unknown' inside lists
    if isinstance(start_dates, list) and any(str(d).lower() == 'unknown' for d in start_dates):
        return 'unknown'
    if isinstance(end_dates, list) and any(str(d).lower() == 'unknown' for d in end_dates):
        return 'unknown'

    # Check for NaNs safely
    if isinstance(start_dates, list):
        if any(pd.isna(d) for d in start_dates):
            return np.nan
    elif pd.isna(start_dates):
        return np.nan

    if isinstance(end_dates, list):
        if any(pd.isna(d) for d in end_dates):
            return np.nan
    elif pd.isna(end_dates):
        return np.nan

    # Calculate total years of experience
    total_years = 0
    for start, end in zip(start_dates, end_dates):
        try:
            start_dt = datetime.strptime(str(start).strip(), '%Y-%m')
            end_dt = datetime.strptime(str(end).strip(), '%Y-%m')
            total_years += (end_dt - start_dt).days / 365.25
        except Exception:
            return np.nan  # Invalid date format

    return round(total_years, 2) if total_years != 0 else np.nan

df['experience_years'] = df.apply(
    lambda row: calculate_experience(row['start_dates'], row['end_dates']),
    axis=1
)


In [716]:
# Count how many 'unknown' strings are in the experience_years column
unknown_experience_count = (df['experience_years']=='unknown').sum()

print("Total 'unknown' in experience_years:", unknown_experience_count)


Total 'unknown' in experience_years: 788


now remove the 'start_dates' and 'end_dates' columns, because they are no longer useful

In [719]:
df = df.drop(['start_dates', 'end_dates'], axis=1)

'start_dates' and 'end_dates' columns has cleaned. and now the new column 'experience_years' ready to go!
total 'unknown' s are 788 and total 'nan's 112. 